In [129]:
# import dependencies
import sys
import pandas as pd
import hashlib  #hashing library
import time   #produce time stamp
import json   
import requests #request info from the API

In [194]:
def hash_gen():
    #Generating the Hash
    m = hashlib.md5()

    ts = str(time.time())   #creates time stamp as string
    ts_byte = bytes(ts, 'utf-8')  #converts the timestamp into a byte 
    m.update(ts_byte)  # add the timestamp to hash
    m.update(b"8c6e53313bb343e03a9b8d2a6d3da0754415a07b") #add the private key to 
        #the hash in byte format
    m.update(b"6040b6d7af5b870e8309d4fbd15a88d2") #add the public key to 
        #the hash in byte format
    hasht = m.hexdigest()    #Marvel requires the string to be in hex(mentioned nowhere)
    return hasht

In [158]:
def API_call(api_key, hasht, nameStartsWith, offset):
    #constructing the query

    base_url = "https://gateway.marvel.com"  #base url
    query = "/v1/public/characters" +"?"  #Query for all characters

    #Actual query look like:
    #query_url = base_url + query +"nameStartsWith=spider&limit=10&"+"ts=" + ts+ "&apikey=" + api_key + "&hash=" + hasht
    #print(query_url) 
    
    query_url = base_url + query
    ts = str(time.time())
    payload = {
        'nameStartsWith':nameStartsWith,
        'limit':100,
        'ts':ts,
        'apikey':api_key,
        'hash':hasht,
        'offset':offset
    } # all the parameters for the query

    #Making the API request and receiving info back as a json
    data = requests.get(query_url, params=payload).json()
    # json_obj = json.dumps(data, indent=4) # can be further stored as json dump
    
    try:
        #Storing the data to a dataframe
        df = pd.DataFrame(data)
        df_norm = pd.json_normalize(df["data"]["results"])  #Breaks down keys to the basic form
        df_select = df_norm[["id", "name", "comics.available", "series.available", "stories.available", "events.available"]]
        print(nameStartsWith, "queried characters: ", data["data"]["count"])
        return df_select, df["data"]["count"]
    
    except:
        if data["code"] == 200:
            print(nameStartsWith, "queried characters: 0")
        else:
            print(data)
        return None, 0

In [176]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2"
df_ = pd.DataFrame()
offset=0
c=0
while (c==0 and offset==0) or c==100: 
    df_s, c = API_call(api_key, hash_gen(), '%', offset)
    df_ = pd.concat([df_, df_s])
    offset=offset+100


% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  100
% queried characters:  62


In [178]:
df = df_
df.head()

,id,name,comics.available,series.available,stories.available,events.available
0,1011334,3-D Man,12,3,21,1
1,1017100,A-Bomb (HAS),4,2,7,0
2,1009144,A.I.M.,51,34,54,0
3,1010699,Aaron Stack,14,3,27,0
4,1009146,Abomination (Emil Blonsky),55,27,65,1


In [185]:
import numpy as np

In [182]:
df_condition = df["comics.available"].apply(lambda x: eval(f"{x} {'=='} {'10'}"))
df[df_condition]




# print(df["comics.available"].apply(f"{'comics.available'} {'=='} {'10'}"))

,id,name,comics.available,series.available,stories.available,events.available
46,1011396,Angel (Thomas Halloway),10,5,9,0
72,1011236,Bride of Nine Spiders (Immortal Weapons),10,2,14,1
17,1009225,Captain Stacy,10,4,8,0
19,1011052,Cardiac,10,4,10,1
5,1010907,Dark Beast,10,5,10,2
7,1011387,Dark X-Men,10,3,17,1
88,1009292,Emplate,10,5,9,1
92,1010445,Energizer,10,4,17,0
19,1011218,Harry Osborn (Ultimate),10,6,7,0
46,1010812,John Jameson,10,6,9,0


In [ ]:

x = lambda col, filter, value: df[eval(f"{df[col]} {filter} {value}")] 
print(x("comics.available", '>', '10'))

In [144]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2"

df_s, c = API_call(api_key, hash_gen(), 's', 200)

s queried characters:  2


In [146]:
c

2

In [145]:
df_s

,id,name,comics.available,series.available,stories.available,events.available
0,1009642,Sym,2,1,3,0
1,1009643,Synch,18,12,18,1


In [140]:
df_s

,id,name,comics.available,series.available,stories.available,events.available
0,1011070,Slayback,1,1,1,0
1,1009601,Sleeper,6,4,6,0
2,1010761,Sleepwalker,13,8,13,1
3,1009602,Slipstream,5,2,5,0
4,1010874,Slyde,1,1,1,0
...,...,...,...,...,...,...
95,1011017,Sway,1,1,1,0
96,1009640,Switch,4,1,5,0
97,1017852,Sword Master (Sword Master),25,7,25,0
98,1009641,Swordsman,60,25,56,4


In [138]:
df_s

,id,name,comics.available,series.available,stories.available,events.available
0,1009552,S.H.I.E.L.D.,165,72,196,1
1,1009553,Sabra,21,13,21,0
2,1009554,Sabretooth,309,113,326,8
3,1010969,Sabretooth (Age of Apocalypse),21,7,21,1
4,1010970,Sabretooth (House of M),1,1,1,1
...,...,...,...,...,...,...
95,1011157,Skreet,5,3,4,1
96,1009599,Skrulls,132,62,141,5
97,1015002,Skrulls (Ultimate),2,2,2,0
98,1009600,Skullbuster (Cylla Markham),3,3,3,0


In [134]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2"

df_9 = API_call(api_key, hash_gen(), '9')

9 queried characters: 0


In [119]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2" 
df_3 = API_call(api_key, hash_gen(), '3')

3 queried characters:  1


In [121]:
pd.concat([df_9, df_3])

,id,name,comics.available,series.available,stories.available,events.available
0,1011334,3-D Man,12,3,21,1


In [151]:
def All_calls(api_key):
    #All marvel characters fetched by going through every ascii characters
    i = 65
    df_final = pd.DataFrame()
    while i<69:
        offset=0
        c=0
        while (c==0 and offset==0) or c==100: 
            df_select, c = API_call(api_key, hash_gen(), chr(i), offset)
            df_final = pd.concat([df_final, df_select])   #appending every call data
            offset = offset+100  
        i=i+1
    return df_final

In [152]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2" 
df_final = All_calls(api_key)

A queried characters:  84
B queried characters:  100
B queried characters:  1
C queried characters:  100
C queried characters:  4
D queried characters:  80


In [153]:
df_final.head()

,id,name,comics.available,series.available,stories.available,events.available
0,1017100,A-Bomb (HAS),4,2,7,0
1,1009144,A.I.M.,51,34,54,0
2,1010699,Aaron Stack,14,3,27,0
3,1009146,Abomination (Emil Blonsky),55,27,65,1
4,1016823,Abomination (Ultimate),2,2,3,0


In [154]:
df_final.shape

(369, 6)

In [126]:
df_rm_dup = df_final.drop_duplicates(subset=["id"]) #remove duplicates

In [127]:
df_rm_dup.shape

(1399, 6)

In [128]:
df_rm_dup.to_csv("./marvel_data.csv", index=False) #saving the dataset

# MARVEL FILTER

In [17]:
df = pd.read_csv("./marvel_data.csv")   #opening the dataset

In [18]:
df.head()

,Unnamed: 0,id,name,comics.available,series.available,stories.available,events.available
0,0,1011334,3-D Man,12,3,21,1
1,1,1017100,A-Bomb (HAS),4,2,7,0
2,2,1009144,A.I.M.,51,34,54,0
3,3,1010699,Aaron Stack,14,3,27,0
4,4,1009146,Abomination (Emil Blonsky),55,27,65,1


In [19]:
def filter(df, op_column, op, value, display_column):
    #make sql like where queries by passing in the column, value , comparision operator
    #and select the columns you want to be displayed
    df_show = pd.DataFrame()
    match op:
        case '>':
            df_show = df[df[op_column]>value][display_column]
            return df_show
        case '<':
            df_show = df[df[op_column]<value][display_column]
            return df_show
        case '=':
            df_show = df[df[op_column]==value][display_column]
            return df_show
        case '>=':
            df_show = df[df[op_column]>=value][display_column]
            return df_show
        case '<=':
            df_show = df[df[op_column]<=value][display_column]
            return df_show

SyntaxError: invalid syntax (<ipython-input-19-bdf780bf44b4>, line 5)

In [48]:
df_show = filter(df, "comics.available", '=', 4, ["id", "name"])

In [49]:
df_show

,id,name
1,1017100,A-Bomb (HAS)
19,1011136,Air-Walker (Gabriel Lan)
20,1011176,Ajak
41,1010673,Amphibian (Earth-712)
93,1011354,Barracuda
195,1011139,Cammi
219,1010143,Caretaker
250,1012065,Chthon
257,1011286,Cobalt Man
279,1014991,Crusher Hogan (Ultimate)


In [166]:
def filter(df, size, op_column, op, value, display_column):
    #make sql like where queries by passing in the column, value , comparision operator
    #and select the columns you want to be displayed
    df_show = pd.DataFrame()
    match op:
        case '>':
            df_show = df[df[op_column]>value][display_column]
            return df_show
        case '<':
            df_show = df[df[op_column]<value][display_column]
            return df_show
        case '=':
            df_show = df[df[op_column]==value][display_column]
            return df_show
        case '>=':
            df_show = df[df[op_column]>=value][display_column]
            return df_show
        case '<=':
            df_show = df[df[op_column]<=value][display_column]
            return df_show

In [169]:
df_show = filter(df_final, 2, ["comics.available", "name"], ['=', '='], [4, 'Aaron Stack'], ["id", "name"])

TypeError: filter() takes 5 positional arguments but 6 were given